In [ ]:
import sys
import pathlib
from os import chdir
while not str(pathlib.Path.cwd().name).lower().startswith('process-improve'):
    chdir(pathlib.Path.cwd().parents[0])
basecwd =pathlib.Path.cwd()
sys.path.insert(0, str(basecwd))
assert basecwd.exists()
%load_ext autoreload
%autoreload 2
import plotly.graph_objects as go
import random
import math
import seaborn as sns
from functools import partial
import seaborn as sns
import pandas as pd
from process_improve.batch.plotting import plot_multitags

In [ ]:
url = "https://raw.githubusercontent.com/kgdunn/process-improve/main/process_improve/datasets/batch/dryer.csv"
dataset = pd.read_csv(url)

from process_improve.batch.data_input import melted_to_dict
from process_improve.batch.plotting import plot_multitags
df_dict = melted_to_dict(dataset, batch_id_col="batch_id")

In [ ]:
batch1 = df_dict[list(df_dict.keys())[0]]
tags_to_plot = batch1.columns.to_list()
tags_to_plot.remove('batch_id')
tags_to_plot.remove('ClockTime')

In [ ]:
settings = {}
settings["nrows"] = 1
settings["x_axis_label"] = "Time sample"
settings["show_legend"] = True
settings["colour_map"] = partial(sns.color_palette, "husl")  # hls; but husl is nicer
settings["animate"] = True
settings["animate_batches_to_highlight"] = [1,2,34, 4]
settings["animate_show_slider"] = True
settings["animate_show_pause"] = False
settings["animate_n_frames"] = 10
settings["animate_framerate_milliseconds"] = 4000
settings["animate_slider_vertical_offset"] = 1.4
settings["animate_line_width"] = 5
settings["default_line_width"] = 0.5

settings['html_image_height' ] = 600

fig=plot_multitags(
    df_dict,
    time_column="ClockTime",
    tag_list=tags_to_plot[0:3],
    settings=settings,
)
fig.show()

In [ ]:
fig["frames"][9]['data'][-9]

In [ ]:
fig['data'][-3]

In [ ]:
assert False

# Proof-of-concept code

In [ ]:
url = "https://raw.githubusercontent.com/kgdunn/process-improve/main/process_improve/datasets/batch/dryer.csv"
dataset = pd.read_csv(url)

from process_improve.batch.data_input import melted_to_dict
from process_improve.batch.plotting import plot_multitags
df_dict = melted_to_dict(dataset, batch_id_col="batch_id")

In [ ]:
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
settings = {}
fig = go.Figure()
batch1 = df_dict[list(df_dict.keys())[0]]

redraw = True

time_column = None
tag_list=tags_to_plot
batch_list = df_dict.keys()

settings["ncols"] = 5
settings["nrows"] = 2
settings["show_legend"] = True
settings["moving_line_width"] = 4
specs = [[{"type": "scatter"}] * int(settings["ncols"])] * int(settings["nrows"])

fig.set_subplots(
    rows=settings["nrows"],
    cols=settings["ncols"],
    shared_xaxes="all",
    shared_yaxes=False,
    start_cell="top-left",
    vertical_spacing=0.2 / settings["nrows"],
    horizontal_spacing=0.2 / settings["ncols"],
    subplot_titles=tag_list,
    specs=specs,
);

# Background/starting visualization: grey lines which get added to
for batch_id, batch_df in df_dict.items():
    if batch_id not in batch_list:
        continue
    # Time axis values
    if time_column in batch_df.columns:
        time_data = batch_df[time_column]
    else:
        time_data = list(range(batch_df.shape[0]))

    row = col = 1
    for tag in tag_list:
        trace = go.Scatter(
            x=time_data,
            y=batch_df[tag],
            name=batch_id,
            mode="lines",
            
            hovertemplate="Time: %{x}\ny: %{y}",
            line={'width': 0.5, 'color': '#999999'}, # later: add golden batches as yellow 
            legendgroup=batch_id,
            #showlegend=settings["show_legend"] if tag == tag_list[0] else False,
            showlegend=False
        )
        fig.add_trace(trace, row=row, col=col)

        col += 1
        if col > settings["ncols"]:
            row += 1
            col = 1

# Base slider
slider_baseline_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Hours:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}


def get_rgba_from_triplet(incolour, alpha=1, as_string=False):
    """
    Convert the input colour triplet (list) to a Plotly rgba(r,g,b,a) string if
    `as_string` is True. If `False` it will return the list of 3 integer RGB
    values.

    E.g.    [0.9677975592919913, 0.44127456009157356, 0.5358103155058701] -> 'rgba(246,112,136,1)'
    """
    assert len(incolour) == 3
    colours = [max(0, int(math.floor(c * 255))) for c in incolour]
    if as_string:
        return f"rgba({colours[0]},{colours[1]},{colours[2]},{float(alpha)})"
    else:
        return colours
n_colours = len(df_dict)
random.seed(13)
colours = list(sns.husl_palette(n_colours))
random.shuffle(colours)
colours = [get_rgba_from_triplet(c, as_string=True) for c in colours]
colour_assignment = {
    key: dict(width=settings["moving_line_width"], color=val)
    for key, val in zip(list(df_dict.keys()), colours)
}

from typing import List, Dict
def generate_one_frame(df_dict, fig, tag_list, index, time_column, batch_ids, show_legend=False, add_hovertemplate=False, max_columns=0) -> List[Dict]:
    """
    Returns a list of dictionaries.
    Each entry in the list is for each subplot; in the order of the subplots. 
    Since each subplot is a tag, we need the `tag_list` as input
    """
    
    output= []
    row = col = 1    
    for tag in tag_list:      
           
        for batch_id in batch_ids:    
            # These 4 lines are duplicated from the outside function
            if time_column in df_dict[batch_id].columns:
                time_data = df_dict[batch_id][time_column]
            else:
                time_data = list(range(df_dict[batch_id].shape[0]))

            output.append(
                go.Scatter(
                    x=time_data[0:index],
                    y=df_dict[batch_id][tag][0:index],
                    name=batch_id,
                    mode="lines",
                    hovertemplate="Time: %{x}\ny: %{y}",
                    line=colour_assignment[batch_id],
                    legendgroup=batch_id,
                    showlegend=show_legend, #settings["show_legend"] if tag == tag_list[0] else False,
                    xaxis=fig.get_subplot(row, col)[1]['anchor'],
                    yaxis=fig.get_subplot(row, col)[0]['anchor'],
                )
            )
        
        col += 1
        if col > max_columns:
            row += 1
            col = 1
    return output
    
        
        


frames = []
slider_steps = []
frame_settings = {
    'frame': {'duration': 0, 'redraw': False},
    'mode': 'immediate',
    'transition': {'duration': 0}
}
n_frames = 10
last_time_step_index = 200
batch_ids = [48, 34, 41]


#Frames: a list of dicts. Each dict has the keys: 'name', 'data', and 'traces'.#
# frames = [dict(name=k,
#                data=[dict(y=10*np.random.rand(6)),
#                    dict(y=1.5+7.5*np.random.rand(20),
#                         marker=dict(color=np.random.choice(pl_colors))),
#                    dict(y=10*np.random.rand(6),
#                        )
#                    ],
#                traces=[0,1,2]) for k in range(10)]

# 'data' value is a list of dicts, 
#       each dict contains only the original trace attributes that are updated by this frame. 
#  In this example the y-values in all tree basic traces are updated, as well as the marker color in the trace of index 1.


for index in np.linspace(0, last_time_step_index, n_frames): # todo: pick the longest batch
    # TO OPTIMIZE: add hover template only on the last iteration
    # TO OPTIMIZE: can you add only the incremental new piece of animation?
    index = int(np.floor(index))
    frame_name = f"{index}"  # this is the link with the slider and the animation in the play button
    

    frames.append(
        go.Frame(
            data=generate_one_frame(df_dict, fig, tag_list, index+1, time_column, batch_ids=batch_ids, show_legend=True, add_hovertemplate=False, max_columns=settings['ncols']),
            #traces = list(range(len(tag_list))) ,
            name =frame_name
        )
    )

    # fig_dict["layout"]['sliders'][0]['steps'][0].keys()  # dict_keys(['args', 'label', 'method'])
    slider_dict = dict(
        args=[
            [frame_name], # hour
            frame_settings,
        ],
        label=f'{index}',
        method= "animate",
    )
    
    slider_steps.append(slider_dict)
    
    
# Bring the things together    
fig.frames=frames
slider_baseline_dict['steps'] = slider_steps
fig["layout"]["sliders"] = [slider_baseline_dict]

# fig["layout"]['sliders']   # list of 1 element
# fig_dict["layout"]['sliders'][0].keys()   # dict_keys(['active', 'yanchor', 'xanchor', 'currentvalue', 'transition', 'pad', 'len', 'x', 'y', 'steps'])
# len(fig_dict["laayout"]['sliders'][0]['steps'])  # list of n-frames 
# fig_dict["layout"]['sliders'][0]['steps'][0].keys()  # dict_keys(['args', 'label', 'method'])
# len(fig_dict["layout"]['sliders'][0]['steps'][0]['args'])   # list of 2 elements:   [ ['1952'], dict <-- see next]
# fig_dict["layout"]['sliders'][0]['steps'][0]['args'][1]   # dict: {'frame': {'duration': 300, 'redraw': True},  'mode': 'immediate', 'transition': {'duration': 300}}

button_play = dict(
    label='Play',
    method='animate',
    args=[
        None, 
        dict(
            frame=dict(
                duration=0, 
                redraw=False
            ),
            transition=dict(duration=30, easing="quadratic-in-out"),
            fromcurrent=True,
            mode='immediate'
        )
    ]
)

button_pause = dict(
    label='Pause',
    method='animate',
    args=[
        # https://plotly.com/python/animations/ 
        # Note the None is in a list!
        [None],  
        dict(
            frame=dict(
                duration=0, 
                redraw=False
            ),
            transition=dict(duration=0),
            mode='immediate'
        )
    ]
)

fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=0,
            x=1.05,
            xanchor='left',
            yanchor='bottom',
            buttons=[button_play, button_pause],
        )
    ],
    width=1200, 
    height=800
)
                              
# fig.update_layout(yaxis2_range=[0, 5.5], yaxis2_autorange=True)  
fig.show()


In [ ]:
margin_dict = dict(l=10, r=10, b=5, t=80)  # Defaults: l=80, r=80, t=100, b=80

from plotly.offline import plot as plotoffline

config = {
    "scrollZoom": True,
    "displaylogo": False,
    "editable": False, 
    "showLink": False,
}

plotly_div_string = plotoffline(
    fig,
    config=config,
    output_type="div",
    include_plotlyjs="cdn",
    include_mathjax="cdn",
    validate=True,
)
len(plotly_div_string    )/1024/1024

In [ ]:
fig2 = make_subplots(
    rows=1, cols=2, subplot_titles=('Title1', 'Title2'),
    horizontal_spacing=0.051
)

fig2.add_trace(go.Scatter(x=np.arange(10),
                         y=1+3*np.random.rand(10),
                        marker_size=6), row=1, col=1 )

fig2.add_trace(go.Scatter(x=np.arange(10),
                         y=1+3*np.random.rand(10),
                        marker_size=6), row=1, col=2) 


#traces=[0, 1, 2]` in the frame definition makes the difference: it tells that 
#the traces of index 0, 1 from the subplot(1,1), are unchanged, and we only ensure their visibility in each #frame (because neither x nor y are modified)
#while the trace 2 from the subplot(1,2) is animated, because the y-values are changed. 

n_frames = 10
frames_simple =[
    go.Frame(
        data=[
            go.Scatter(
                y=k/n_frames*3.5+0.01*np.random.rand(10)
            ), 
            go.Scatter(
                y=k/n_frames*3.5+0.01*np.random.rand(10)
            )
        ],
        traces=[0,1],
        name=str(k)
    ) for k in range(n_frames)
]

fig2.frames=frames_simple

sliders_dict_baseline = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 12},
        "prefix": "Step:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

frame_settings = {
    'frame': {'duration': 0, 'redraw': False},
    'mode': 'immediate',
    'transition': {'duration': 0}
}
for index in range(n_frames):
    sub_step = {
        "args": [
            [str(index)],
        {"frame": {"duration": 300, "redraw": redraw},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": str(index),
        "method": "animate"}
    
    
    
    sliders_dict_baseline["steps"].append(sub_step)


sliders_dict_baseline["steps"] = slider_steps
fig2["layout"]["sliders"] = [sliders_dict_baseline]

button = dict(
             label='Play',
             method='animate',
             args=[None, dict(frame=dict(duration=500, redraw=False), 
                              transition=dict(duration=0),
                              fromcurrent=True,
                              mode='immediate')])
fig2.update_layout(updatemenus=[
    dict(
        type='buttons',
        showactive=False,
        direction="left",
        pad= {"r": 10, "t": 87},
        y=0,
        x=1.05,
        xanchor='left',
        yanchor='bottom',
        buttons=[button] 
    )
], width=800, height=500)
fig2.update_layout(yaxis1_range=[0, 3.5], yaxis1_autorange=False)                              
fig2.update_layout(yaxis2_range=[0, 3.5], yaxis2_autorange=False)  
fig2

In [ ]:

import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

fig = dict(
    layout = dict(width=900, height=500,
        xaxis1 = {'domain': [0.0, 0.44], 'anchor': 'y1', 'title': '1', 'range': [-2.25, 3.25]},
        yaxis1 = {'domain': [0.0, 1.0], 'anchor': 'x1', 'title': 'y', 'range': [-1, 11]},
        xaxis2 = {'domain': [0.56, 1.0], 'anchor': 'y2', 'title': '2', 'range': [-2.25, 3.25]},
        yaxis2 = {'domain': [0.0, 1.0], 'anchor': 'x2', 'title': 'y', 'range': [-1, 11]},
        title  = '',
        margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
    ),

    data = [
        {'type': 'scatter', # This trace is identified inside frames as trace 0
         'name': 'f1', 
         'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 
         'y': [  4,   1,   1, 1,   4,   9], 
         'hovertemplate': "Time: %{x}\ny: %{y}", 
         'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}},
         'line': {'color': 'rgba(255,79,38,1.000000)'}, 
         'mode': 'markers+lines', 
         'fillcolor': 'rgba(255,79,38,0.600000)', 
         'legendgroup': 'f1',
         'showlegend': True, 
         'xaxis': 'x1', 'yaxis': 'y1'},
        {'type': 'scatter', # This trace is identified inside frames as trace 1
        'name': 'f12', 
        'x': -1.5+4.25*np.random.rand(20),
        'y': 1.5+7.5*np.random.rand(20),
        'mode': 'markers',
        'marker': {'size': 10, 'color':'blue'},
        'xaxis': 'x1', 'yaxis': 'y1'},
        {'type': 'scatter', # # This trace is identified inside frames as trace 2
         'name': 'f2', 
         'x': [-2.  , -1.  ,  0.01,  1.  ,  2.  ,  3.  ], 
         'y': [  2.5,   1,   1, 1,   2.5,   1], 
         'hoverinfo': 'name+text', 
         'marker': {'opacity': 1.0, 'symbol': 'circle', 'line': {'width': 0, 'color': 'rgba(50,50,50,0.8)'}}, 
         'line': {'color': 'rgba(79,102,165,1.000000)'}, 
         'mode': 'markers+lines', 'fillcolor': 'rgba(79,102,165,0.600000)', 
         'legendgroup': 'f2', 'showlegend': True, 'xaxis': 'x2', 'yaxis': 'y2'},
    ]

    
)
frames = [dict(name=k,
               data=[dict(y=10*np.random.rand(6)),
                   dict(y=1.5+7.5*np.random.rand(20),
                        marker=dict(color=(0.5, 0.5, 0.5))),
                   dict(y=10*np.random.rand(6),
                       )
                   ],
               traces=[0,1,2]) for k in range(10)]

updatemenus = [dict(type='buttons',
                    buttons=[dict(label='Play',
                                  method='animate',
                                  args=[[f'{k}' for k in range(10)], 
                                         dict(frame=dict(duration=500, redraw=False), 
                                              transition=dict(duration=0),
                                              easing='linear',
                                              fromcurrent=True,
                                              mode='immediate'
                                                                 )]),
                             dict(label='Pause',
                                  method='animate',
                                  args=[None,
                                        dict(frame=dict(duration=500, redraw=False), 
                                             transition=dict(duration=0),
                                             easing='linear',
                                             fromcurrent=True,
                                             mode='immediate' )])],
                    direction= 'left', 
                    pad=dict(r= 10, t=85), 
                    showactive =True, x= 0.1, y= 0, xanchor= 'right', yanchor= 'top')
            ]
sliders = [{'yanchor': 'top',
            'xanchor': 'left', 
            'currentvalue': {'font': {'size': 16}, 'prefix': 'Frame: ', 'visible': True, 'xanchor': 'right'},
            'transition': {'duration': 500.0, 'easing': 'linear'},
            'pad': {'b': 10, 't': 50}, 
            'len': 0.9, 'x': 0.1, 'y': 0, 
            'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': False},
                                      'transition': {'duration': 0, 'easing': 'linear'}}], 
                       'label': k, 'method': 'animate'} for k in range(10)       
                    ]}]     
fig.update(frames=frames),
fig['layout'].update(updatemenus=updatemenus,
          sliders=sliders)
iplot(fig)

In [ ]:
fig["frames"][4]

In [ ]:
fig["data"]